In [ ]:
!pip install datasets


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/dataset.zip /content/
!cp /content/drive/MyDrive/dataset.zip /content/
!unzip -q dataset.zip -d dataset


In [ ]:
!git clone https://github.com/BioMikeUkr/dl-first-task.git

Cloning into 'dl-first-task'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 19 (delta 6), reused 13 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 9.74 KiB | 9.74 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
import json
import random
random.seed(42)

data = json.load(open("data.json", "rt"))
random.shuffle(data)
print(len(data))
data = data[:int(len(data)*0.95)]
print(len(data))

3047
2894


In [ ]:
3047 - 2894

153

In [ ]:
from transformers import Trainer, TrainingArguments
from src.model import ClipSegMultiClassModel
from src.config import ClipSegMultiClassConfig
from src.data_processing import SingleClassSegmentationDataset, SegmentationCollator

class_labels = ["background", "Pig", "Horse", "Sheep"]

config = ClipSegMultiClassConfig(
    class_labels=class_labels,
    label2color={
        0: [0, 0, 0],
        1: [255, 0, 0],
        2: [0, 255, 0],
        3: [0, 0, 255]
    },
    model="CIDAS/clipseg-rd64-refined"
)

model = ClipSegMultiClassModel(config).to("cuda")

train_dataset = SingleClassSegmentationDataset(
    dataset=data,
    class_labels=class_labels,
    image_size=352
)

collator = SegmentationCollator(model.processor, class_labels)

training_args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=64,
    learning_rate=1e-4,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=5,
    save_strategy="epoch",
    evaluation_strategy="no",
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=None,
    data_collator=collator,
)

trainer.train()

model.save_pretrained("./trained_clipseg_multiclass")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-68bb663889ff>:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding', 'truncation'
  return self.preprocess(images, **kwargs)


Step,Training Loss
5,1.785800
10,1.354100
15,1.150800
20,1.023800
25,0.906000
30,0.878100
35,0.801100
40,0.766400
45,0.745800
50,0.717100


/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding', 'truncation'
  return self.preprocess(images, **kwargs)
/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding', 'truncation'
  return self.preprocess(images, **kwargs)
/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding', 'truncation'
  return self.preprocess(images, **kwargs)


KeyboardInterrupt: 

In [ ]:
from transformers import Trainer, TrainingArguments
from src.model import ClipSegMultiClassModel
from src.config import ClipSegMultiClassConfig
from src.data_processing import SingleClassSegmentationDataset, SegmentationCollator


model = ClipSegMultiClassModel.from_pretrained("trained_clipseg_multiclass")
model.push_to_hub("BioMike/clipsegmulticlass_v1", token="hf_token")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from PIL import Image
import torch


model.eval()
model = ClipSegMultiClassModel.from_pretrained("trained_clipseg_multiclass")
image = Image.open("pigs.jpg").convert("RGB")
mask = model.predict(image)
print(mask.shape)

import numpy as np
import matplotlib.pyplot as plt

def visualize_mask(mask_tensor: torch.Tensor, label2color: dict):

    if mask_tensor.dim() == 3:
        mask_tensor = mask_tensor.squeeze(0)

    mask_np = mask_tensor.cpu().numpy().astype(np.uint8)  # [H, W]

    h, w = mask_np.shape
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)

    for class_idx, color in label2color.items():
        color_mask[mask_np == class_idx] = color

    return color_mask


color_mask = visualize_mask(mask, config.label2color)

plt.imshow(color_mask)
plt.axis("off")
plt.title("Predicted Segmentation Mask")
plt.show()

# Image.fromarray(color_mask).save("predicted_mask.png")

NameError: name 'Image' is not defined